In [ ]:
import numpy as np

# 활성화 함수 (시그모이드) 및 그 미분
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

class MLP:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        """
        MLP 초기화
        :param input_size: 입력층 뉴런 수
        :param hidden_size: 은닉층 뉴런 수
        :param output_size: 출력층 뉴런 수
        :param learning_rate: 학습률
        """
        # 가중치 초기화 (u: 입력층 -> 은닉층, v: 은닉층 -> 출력층)
        self.u = np.random.randn(input_size, hidden_size) * 0.01  # u 초기화
        self.v = np.random.randn(hidden_size, output_size) * 0.01  # v 초기화
        
        # 바이어스 초기화
        self.b_u = np.zeros((1, hidden_size))  # 은닉층 바이어스
        self.b_v = np.zeros((1, output_size))  # 출력층 바이어스
        
        self.learning_rate = learning_rate

    def forward(self, x):
        """
        순전파 (Forward Pass)
        :param x: 입력 데이터 (1, input_size)
        :return: 은닉층 출력 z, 최종 출력 o
        """
        # 입력층 -> 은닉층
        self.z_sum = np.dot(x, self.u) + self.b_u  # z_sum = x * u + b_u
        self.z = sigmoid(self.z_sum)  # z = τ(z_sum)
        
        # 은닉층 -> 출력층
        self.o_sum = np.dot(self.z, self.v) + self.b_v  # o_sum = z * v + b_v
        self.o = sigmoid(self.o_sum)  # o = τ(o_sum)
        
        return self.z, self.o

    def backward(self, x, t, z, o):
        """
        역전파 (Backward Pass)
        :param x: 입력 데이터
        :param t: 타겟 값
        :param z: 은닉층 출력
        :param o: 최종 출력
        """
        # 출력층에서의 오차 (δ_k)
        delta_o = (o - t) * sigmoid_derivative(self.o_sum)  # δ_k = (o_k - t_k) * τ'(o_sum_k)
        
        # 은닉층에서의 오차 (δ_j)
        delta_z = np.dot(delta_o, self.v.T) * sigmoid_derivative(self.z_sum)  # δ_j = (Σ δ_k * v_jk) * τ'(z_sum_j)
        
        # 가중치 v와 바이어스 b_v 업데이트
        v_grad = np.dot(z.T, delta_o)  # ∂E/∂v = z * δ_k
        self.v -= self.learning_rate * v_grad  # v -= η * ∂E/∂v
        self.b_v -= self.learning_rate * np.sum(delta_o, axis=0, keepdims=True)  # b_v -= η * δ_k
        
        # 가중치 u와 바이어스 b_u 업데이트
        u_grad = np.dot(x.T, delta_z)  # ∂E/∂u = x * δ_j
        self.u -= self.learning_rate * u_grad  # u -= η * ∂E/∂u
        self.b_u -= self.learning_rate * np.sum(delta_z, axis=0, keepdims=True)  # b_u -= η * δ_j

    def train(self, X, T, epochs=1000, stop_condition=1e-4):
        """
        MLP 학습
        :param X: 훈련 데이터 (N, input_size)
        :param T: 타겟 데이터 (N, output_size)
        :param epochs: 최대 반복 횟수
        :param stop_condition: 종료 조건 (오차 임계값)
        """
        for epoch in range(epochs):
            total_error = 0
            for i in range(len(X)):
                # 순전파
                x = X[i:i+1]  # (1, input_size)
                t = T[i:i+1]  # (1, output_size)
                z, o = self.forward(x)
                
                # 손실 계산 (평균 제곱 오차)
                error = 0.5 * np.sum((o - t) ** 2)
                total_error += error
                
                # 역전파
                self.backward(x, t, z, o)
            
            # 평균 오차 출력
            avg_error = total_error / len(X)
            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Average Error: {avg_error}")
            
            # 종료 조건 확인
            if avg_error < stop_condition:
                print(f"종료 조건 만족 (오차 < {stop_condition})")
                break

    def predict(self, x):
        """
        예측
        :param x: 입력 데이터
        :return: 예측값
        """
        _, o = self.forward(x)
        return o

# 테스트  
if __name__ == "__main__":
    # XOR 문제 데이터
    X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])  # 입력
    T = np.array([[0], [1], [1], [0]])  # 타겟 (XOR)
    
    # MLP 초기화 (입력층: 2, 은닉층: 4, 출력층: 1)
    mlp = MLP(input_size=2, hidden_size=4, output_size=1, learning_rate=0.1)
    
    # 학습
    mlp.train(X, T, epochs=10000, stop_condition=1e-3)
    
    # 예측
    print("\n예측 결과:")
    for i in range(len(X)):
        x = X[i:i+1]
        pred = mlp.predict(x)
        print(f"입력: {X[i]}, 타겟: {T[i]}, 예측: {pred[0]}")

Epoch 0, Average Error: 0.1265677260685768
Epoch 100, Average Error: 0.1265613941866337
Epoch 200, Average Error: 0.12655599078145963
Epoch 300, Average Error: 0.12655062498369313
Epoch 400, Average Error: 0.1265452969938144
Epoch 500, Average Error: 0.12654000690016495
Epoch 600, Average Error: 0.12653475477934703
Epoch 700, Average Error: 0.12652954069799704
Epoch 800, Average Error: 0.12652436471309808
Epoch 900, Average Error: 0.12651922687221642
Epoch 1000, Average Error: 0.12651412721367017
Epoch 1100, Average Error: 0.12650906576662582
Epoch 1200, Average Error: 0.12650404255112135
Epoch 1300, Average Error: 0.12649905757801105
Epoch 1400, Average Error: 0.12649411084883283
Epoch 1500, Average Error: 0.12648920235559513
Epoch 1600, Average Error: 0.12648433208048346
Epoch 1700, Average Error: 0.12647949999548788
Epoch 1800, Average Error: 0.12647470606195058
Epoch 1900, Average Error: 0.1264699502300363
Epoch 2000, Average Error: 0.1264652324381278
Epoch 2100, Average Error: 0.1